In [36]:
import os
import requests

# Get PDF document path
pdf_path = "prueba.pdf"

# Download PDF
if not os.path.exists(pdf_path):
    print("[INFO] File doesn't exist, downloading...")

    # Enter the URL of the PDF
    url = ""

    # The local filename to save the downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open the file and save it
        with open(filename, "wb") as file:
            file.write(response.content) 
        print(f"[INFO] The file has been download and saved as {filename}")
    else:
        print(f"[INFO] Failed to download the file. Status code: {reponse.status_code}")

else:
    print(f"File {pdf_path} exists.")

File prueba.pdf exists.


In [37]:
import fitz  # Para leer PDFs
import spacy  # Para tokenizar texto
import re  # Para detectar funciones y módulos
from tqdm.auto import tqdm

# Inicializar spaCy para contar tokens
nlp = spacy.blank("es")

def text_formatter(text: str) -> str:
    """Limpia y formatea el texto manteniendo la estructura del código."""
    # Eliminar comentarios y líneas innecesarias
    #text = re.sub(r'//.*', '', text)  # Eliminar comentarios de una sola línea
    #text = re.sub(r'/\*.*?\*/', '', text, flags=re.DOTALL)  # Eliminar comentarios multilinea

    # Reemplazar saltos de línea innecesarios, pero mantener los que separan funciones o bloques
    formatted_text = text.replace("\n", " ").strip()  # Limpiar espacios extra
    formatted_text = ' '.join(formatted_text.split())  # Reducir múltiples espacios a uno solo
    return formatted_text

def count_tokens_spacy(text: str) -> int:
    """Cuenta tokens con spaCy."""
    doc = nlp(text)
    return len(doc)

def extract_functions(text: str) -> list[str]:
    """Extrae nombres de funciones en C# usando expresiones regulares."""
    # Ajustamos el patrón para que también detecte funciones privadas, protegidas, etc.
    pattern = r"\b(?:public|private|protected|internal)\s+(?:override\s+)?(?:void|int|string|bool|double|decimal|float|long|object)\s+(\w+)\s*\("
    return re.findall(pattern, text)  # Lista de funciones encontradas

def extract_module(text: str, current_module: str) -> str:
    """
    Extrae el nombre del módulo basado en patrones de inicio de módulos.
    Si no encuentra uno nuevo, devuelve el módulo actual.
    """
    # Patrón 1: Buscamos en los comentarios /** FORMULARIO NOMBRE **/
    match = re.search(r"/\*\*\s*FORMULARIO\s+([\w\s]+?)\s*\*\*/", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()

    # Patrón 2: Buscamos en etiquetas XML como <row> o <codigoNet>
    match = re.search(r"<(row|codigoNet)>\s*([\w\s]+?)\s*</\1>", text, re.IGNORECASE)
    if match:
        return match.group(2).strip()

    return current_module  # Si no se encuentra un nuevo módulo, mantener el actual

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    current_module = "Desconocido"  # Valor por defecto antes de encontrar el primer módulo
    
    for page_number, page in tqdm(enumerate(doc), total=len(doc)):
        text = page.get_text()
        text = text_formatter(text)
        
        # Extraer el módulo si hay un nuevo marcador
        current_module = extract_module(text, current_module)
        
        # Extraer funciones
        functions = extract_functions(text)

        if functions:  # Solo guardar si hay funciones en la página
            pages_and_texts.append({
                "module": current_module,
                "functions": functions,
                "page_number": page_number,  # Ajusta la numeración si es necesario
                "char_count": len(text),
                "word_count": len(text.split()),
                "token_count": count_tokens_spacy(text),
                "text": text  # Guarda el código completo si es necesario
            })
    
    return pages_and_texts

# 📌 **Ejecutar la extracción en el PDF**
pdf_path = pdf_path
results = open_and_read_pdf(pdf_path)

# Mostrar los primeros resultados
import json
print(json.dumps(results[:2], indent=4, ensure_ascii=False))


  0%|          | 0/52 [00:00<?, ?it/s]

[
    {
        "module": "PEDIDOS",
        "functions": [
            "pAnadirControlDebajo",
            "pAnadirControlDerecha"
        ],
        "page_number": 0,
        "char_count": 1508,
        "word_count": 175,
        "token_count": 285,
        "text": "<row> <CodigoNet> using AhoraAsistentes; using AhoraCore; using AhoraOCX; using AhoraPartes; using AhoraSistema; using static AhoraCore.VBA.Constants; using static AhoraCore.VBA.Conversion; using static AhoraCore.VBA.DateTime; using static AhoraCore.VBA.Strings; using static AhoraCore.VbMsgBoxResult; using static AhoraCore.VbMsgBoxStyle; using System; namespace AhoraScriptsPantalla { /** FORMULARIO PEDIDOS **/ public class AhoraCl_frmPedidos : AhoraOCX.AhoraBaseScript { const int ANCHO_CAPTION = 4; //67; const int ANCHO_FECHAS = 153; //2300; const int ALTO_BUSCADORES = 20; //300; const int DESPLAZA_V = 3; //50; const int DESPLAZA_H = 3; //50; const int ANCHO_BOTONERTEXTOSIMP = 227; //3400; const int ALTO_BOTONERATEXTOSIMP